# 📈 Advanced Trading Model Training - Classification Approach

**Based on Gemini's Recommendations:**
- ✅ Feature engineering with technical indicators
- ✅ Classification problem (UP/DOWN/HOLD)
- ✅ Tree-based models (XGBoost/LightGBM)
- ✅ Direction-focused training

**Goal:** Achieve >65% direction accuracy for profitable trading

In [ ]:
# Install advanced ML packages
!pip install yfinance pandas numpy scikit-learn
!pip install xgboost lightgbm catboost
!pip install ta-lib pandas-ta
!pip install matplotlib seaborn plotly

import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Advanced ML packages installed successfully")

## 📊 Enhanced Data Collection with Market Context

In [ ]:
# Enhanced symbols with market context
SYMBOLS = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA']
MARKET_INDICES = ['^GSPC', '^IXIC', '^DJI']  # S&P 500, NASDAQ, Dow Jones
VOLATILITY_INDEX = ['^VIX']  # Fear & Greed indicator

def fetch_comprehensive_data(symbols, period="2y"):
    """Fetch enhanced market data with context"""
    all_data = {}
    
    # Fetch stock data
    for symbol in symbols:
        print(f"📈 Fetching {symbol}...")
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)
        
        if len(data) > 0:
            all_data[symbol] = data
            print(f"   ✅ {len(data)} data points")
    
    # Fetch market indices for context
    market_data = {}
    for index in MARKET_INDICES + VOLATILITY_INDEX:
        print(f"📊 Fetching {index}...")
        ticker = yf.Ticker(index)
        data = ticker.history(period=period)
        if len(data) > 0:
            market_data[index] = data
            print(f"   ✅ {len(data)} data points")
    
    return all_data, market_data

# Fetch comprehensive dataset
print("🔄 Fetching comprehensive market data...")
stock_data, market_data = fetch_comprehensive_data(SYMBOLS)
print(f"\n📊 Collected data for {len(stock_data)} stocks and {len(market_data)} market indices")

## 🔧 Advanced Feature Engineering - Technical Indicators

In [ ]:
def create_technical_features(df, symbol_name):
    """Create comprehensive technical indicators"""
    data = df.copy()
    
    print(f"🔧 Creating technical features for {symbol_name}...")
    
    # === TREND INDICATORS ===
    # Moving Averages
    data['SMA_5'] = ta.sma(data['Close'], length=5)
    data['SMA_10'] = ta.sma(data['Close'], length=10)
    data['SMA_20'] = ta.sma(data['Close'], length=20)
    data['SMA_50'] = ta.sma(data['Close'], length=50)
    data['EMA_12'] = ta.ema(data['Close'], length=12)
    data['EMA_26'] = ta.ema(data['Close'], length=26)
    
    # MACD
    macd = ta.macd(data['Close'])
    data['MACD'] = macd['MACD_12_26_9']
    data['MACD_Signal'] = macd['MACDs_12_26_9']
    data['MACD_Histogram'] = macd['MACDh_12_26_9']
    
    # === MOMENTUM INDICATORS ===
    # RSI
    data['RSI_14'] = ta.rsi(data['Close'], length=14)
    data['RSI_30'] = ta.rsi(data['Close'], length=30)
    
    # Stochastic
    stoch = ta.stoch(data['High'], data['Low'], data['Close'])
    data['Stoch_K'] = stoch['STOCHk_14_3_3']
    data['Stoch_D'] = stoch['STOCHd_14_3_3']
    
    # Williams %R
    data['Williams_R'] = ta.willr(data['High'], data['Low'], data['Close'])
    
    # === VOLATILITY INDICATORS ===
    # Bollinger Bands
    bb = ta.bbands(data['Close'])
    data['BB_Upper'] = bb['BBU_20_2.0']
    data['BB_Middle'] = bb['BBM_20_2.0']
    data['BB_Lower'] = bb['BBL_20_2.0']
    data['BB_Width'] = (data['BB_Upper'] - data['BB_Lower']) / data['BB_Middle']
    data['BB_Position'] = (data['Close'] - data['BB_Lower']) / (data['BB_Upper'] - data['BB_Lower'])
    
    # Average True Range
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'])
    
    # === VOLUME INDICATORS ===
    # Volume Moving Average
    data['Volume_SMA'] = ta.sma(data['Volume'], length=20)
    data['Volume_Ratio'] = data['Volume'] / data['Volume_SMA']
    
    # On Balance Volume
    data['OBV'] = ta.obv(data['Close'], data['Volume'])
    
    # === PRICE ACTION FEATURES ===
    # Returns
    data['Return_1d'] = data['Close'].pct_change(1)
    data['Return_3d'] = data['Close'].pct_change(3)
    data['Return_5d'] = data['Close'].pct_change(5)
    data['Return_10d'] = data['Close'].pct_change(10)
    
    # Price position within daily range
    data['Price_Position'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])
    
    # Gap analysis
    data['Gap'] = (data['Open'] - data['Close'].shift(1)) / data['Close'].shift(1)
    
    # === RELATIVE STRENGTH FEATURES ===
    # Price relative to moving averages
    data['Price_vs_SMA20'] = data['Close'] / data['SMA_20'] - 1
    data['Price_vs_SMA50'] = data['Close'] / data['SMA_50'] - 1
    
    # Moving average slopes
    data['SMA20_Slope'] = data['SMA_20'].pct_change(5)
    data['SMA50_Slope'] = data['SMA_50'].pct_change(10)
    
    print(f"   ✅ Created {len([col for col in data.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']])} technical features")
    
    return data

def add_market_context(stock_df, market_data, symbol_name):
    """Add market context features"""
    data = stock_df.copy()
    
    print(f"🌐 Adding market context for {symbol_name}...")
    
    # S&P 500 context
    if '^GSPC' in market_data:
        sp500 = market_data['^GSPC']['Close'].reindex(data.index, method='ffill')
        data['SP500_Return'] = sp500.pct_change(1)
        data['Beta_5d'] = data['Return_1d'].rolling(5).corr(data['SP500_Return'])
        data['Relative_Strength'] = (data['Return_1d'] - data['SP500_Return']).rolling(20).mean()
    
    # VIX context (fear & greed)
    if '^VIX' in market_data:
        vix = market_data['^VIX']['Close'].reindex(data.index, method='ffill')
        data['VIX'] = vix
        data['VIX_Change'] = vix.pct_change(1)
        data['Market_Fear'] = (vix > vix.rolling(20).mean()).astype(int)
    
    print(f"   ✅ Added market context features")
    
    return data

# Create enhanced features for all stocks
enhanced_data = {}
for symbol in SYMBOLS:
    if symbol in stock_data:
        # Create technical features
        enhanced = create_technical_features(stock_data[symbol], symbol)
        
        # Add market context
        enhanced = add_market_context(enhanced, market_data, symbol)
        
        # Add symbol identifier
        enhanced['Symbol'] = symbol
        
        enhanced_data[symbol] = enhanced

print(f"\n✅ Enhanced feature engineering completed for {len(enhanced_data)} symbols")
print(f"📊 Total features per symbol: {len(enhanced_data[SYMBOLS[0]].columns)}")

## 🎯 Classification Target Creation

In [ ]:
def create_classification_targets(data, prediction_days=1, threshold=0.01):
    """Create classification targets (UP/DOWN/HOLD)"""
    
    # Calculate future returns
    future_return = data['Close'].shift(-prediction_days) / data['Close'] - 1
    
    # Create classification labels
    conditions = [
        future_return > threshold,    # UP
        future_return < -threshold,   # DOWN
    ]
    choices = [1, -1]  # UP=1, DOWN=-1, HOLD=0 (default)
    
    labels = np.select(conditions, choices, default=0)
    
    return labels, future_return

def prepare_classification_dataset(enhanced_data, prediction_days=1, threshold=0.01):
    """Prepare complete dataset for classification"""
    
    all_features = []
    all_targets = []
    all_returns = []
    metadata = []
    
    print(f"🎯 Creating classification targets (threshold: ±{threshold:.1%})...")
    
    for symbol in SYMBOLS:
        if symbol in enhanced_data:
            data = enhanced_data[symbol].copy()
            
            # Create targets
            targets, future_returns = create_classification_targets(
                data, prediction_days, threshold
            )
            
            # Remove non-feature columns
            feature_cols = [col for col in data.columns 
                          if col not in ['Open', 'High', 'Low', 'Close', 'Volume', 'Symbol']]
            
            features = data[feature_cols].copy()
            
            # Remove rows with NaN values
            valid_mask = ~(features.isna().any(axis=1) | np.isnan(targets))
            
            features_clean = features[valid_mask]
            targets_clean = targets[valid_mask]
            returns_clean = future_returns[valid_mask]
            
            # Store data
            all_features.append(features_clean)
            all_targets.extend(targets_clean)
            all_returns.extend(returns_clean)
            
            # Metadata
            for i in range(len(features_clean)):
                metadata.append({
                    'symbol': symbol,
                    'date': features_clean.index[i],
                    'target': targets_clean[i],
                    'return': returns_clean[i]
                })
            
            # Class distribution for this symbol
            unique, counts = np.unique(targets_clean, return_counts=True)
            class_dist = dict(zip(unique, counts))
            total = sum(counts)
            
            print(f"   📊 {symbol}:")
            print(f"      DOWN (-1): {class_dist.get(-1, 0):3d} ({class_dist.get(-1, 0)/total:.1%})")
            print(f"      HOLD ( 0): {class_dist.get(0, 0):3d} ({class_dist.get(0, 0)/total:.1%})")
            print(f"      UP   ( 1): {class_dist.get(1, 0):3d} ({class_dist.get(1, 0)/total:.1%})")
    
    # Combine all features
    X = pd.concat(all_features, ignore_index=True)
    y = np.array(all_targets)
    returns = np.array(all_returns)
    
    print(f"\n📊 Complete dataset:")
    print(f"   Features shape: {X.shape}")
    print(f"   Targets shape: {y.shape}")
    
    # Overall class distribution
    unique, counts = np.unique(y, return_counts=True)
    total = len(y)
    print(f"\n🎯 Overall class distribution:")
    for label, count in zip(unique, counts):
        label_name = {-1: 'DOWN', 0: 'HOLD', 1: 'UP'}[label]
        print(f"   {label_name}: {count:4d} ({count/total:.1%})")
    
    return X, y, returns, metadata

# Create classification dataset
X, y, future_returns, metadata = prepare_classification_dataset(
    enhanced_data, 
    prediction_days=1, 
    threshold=0.005  # 0.5% threshold for UP/DOWN
)

print(f"\n✅ Classification dataset prepared with {len(X)} samples and {len(X.columns)} features")

## 🏋️ Advanced Model Training - Tree-Based Models

In [ ]:
# Prepare train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Data splits:")
print(f"   Training: {len(X_train)} samples")
print(f"   Testing: {len(X_test)} samples")

# Scale features for some models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models_to_train = {
    'RandomForest': RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    ),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    'LightGBM': lgb.LGBMClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
}

trained_models = {}
model_results = {}

print("\n🏋️ Training advanced classification models...\n")

for name, model in models_to_train.items():
    print(f"🔄 Training {name}...")
    
    # Train model
    if name in ['RandomForest', 'XGBoost', 'LightGBM']:  # Tree-based models
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        probabilities = model.predict_proba(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    
    # Direction accuracy (most important for trading)
    direction_mask = y_test != 0  # Exclude HOLD predictions
    if np.sum(direction_mask) > 0:
        direction_accuracy = accuracy_score(
            y_test[direction_mask], 
            predictions[direction_mask]
        )
    else:
        direction_accuracy = 0
    
    # Store results
    model_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'direction_accuracy': direction_accuracy,
        'predictions': predictions,
        'probabilities': probabilities
    }
    
    trained_models[name] = model
    
    print(f"   ✅ {name}:")
    print(f"      Overall Accuracy: {accuracy:.1%}")
    print(f"      Direction Accuracy: {direction_accuracy:.1%}")
    print(f"      (Direction accuracy is most important for trading)")
    print()

print("✅ All models trained successfully!")

## 📊 Comprehensive Model Evaluation

In [ ]:
def evaluate_trading_performance(y_true, y_pred, returns, model_name):
    """Evaluate model from trading perspective"""
    
    # Basic classification metrics
    accuracy = accuracy_score(y_true, y_pred)
    
    # Direction-only accuracy (excluding HOLD)
    direction_mask = (y_true != 0) & (y_pred != 0)
    direction_accuracy = accuracy_score(y_true[direction_mask], y_pred[direction_mask]) if np.sum(direction_mask) > 0 else 0
    
    # Trading simulation
    trading_returns = []
    correct_predictions = 0
    total_trades = 0
    
    for i in range(len(y_true)):
        if y_pred[i] != 0:  # Only trade on UP/DOWN predictions
            total_trades += 1
            
            # Simulate trading based on prediction
            if y_pred[i] == 1:  # Predicted UP, go long
                trade_return = returns[i]
            else:  # Predicted DOWN, go short
                trade_return = -returns[i]
            
            trading_returns.append(trade_return)
            
            # Check if prediction was correct
            if (y_pred[i] > 0 and returns[i] > 0) or (y_pred[i] < 0 and returns[i] < 0):
                correct_predictions += 1
    
    # Trading metrics
    if len(trading_returns) > 0:
        avg_return = np.mean(trading_returns)
        total_return = np.sum(trading_returns)
        win_rate = correct_predictions / total_trades if total_trades > 0 else 0
        sharpe_ratio = np.mean(trading_returns) / np.std(trading_returns) if np.std(trading_returns) > 0 else 0
    else:
        avg_return = total_return = win_rate = sharpe_ratio = 0
    
    results = {
        'model': model_name,
        'overall_accuracy': accuracy,
        'direction_accuracy': direction_accuracy,
        'total_trades': total_trades,
        'win_rate': win_rate,
        'avg_return_per_trade': avg_return,
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio
    }
    
    return results

# Evaluate all models
print("📊 Comprehensive Model Evaluation\n")
print("=" * 80)

evaluation_results = []

for name, result in model_results.items():
    # Get test returns for this model's predictions
    test_returns = future_returns[-len(y_test):]
    
    eval_result = evaluate_trading_performance(
        y_test, 
        result['predictions'], 
        test_returns, 
        name
    )
    
    evaluation_results.append(eval_result)
    
    print(f"🤖 {name} Performance:")
    print(f"   📈 Overall Accuracy: {eval_result['overall_accuracy']:.1%}")
    print(f"   🎯 Direction Accuracy: {eval_result['direction_accuracy']:.1%} ⭐")
    print(f"   💼 Total Trades: {eval_result['total_trades']}")
    print(f"   🏆 Win Rate: {eval_result['win_rate']:.1%}")
    print(f"   💰 Avg Return/Trade: {eval_result['avg_return_per_trade']:.2%}")
    print(f"   📊 Total Return: {eval_result['total_return']:.2%}")
    print(f"   📈 Sharpe Ratio: {eval_result['sharpe_ratio']:.2f}")
    print("-" * 50)

# Find best model by direction accuracy
best_model = max(evaluation_results, key=lambda x: x['direction_accuracy'])
print(f"\n🏆 BEST MODEL: {best_model['model']} with {best_model['direction_accuracy']:.1%} direction accuracy")

# Check if any model beats 60% direction accuracy threshold
profitable_models = [r for r in evaluation_results if r['direction_accuracy'] > 0.60]
if profitable_models:
    print(f"\n✅ PROFITABLE MODELS ({len(profitable_models)} models > 60% direction accuracy):")
    for model in profitable_models:
        print(f"   🎯 {model['model']}: {model['direction_accuracy']:.1%} direction accuracy")
else:
    print(f"\n⚠️  NO MODELS EXCEED 60% DIRECTION ACCURACY THRESHOLD")
    print(f"   Consider: More feature engineering, longer training periods, or different approaches")

## 🔍 Feature Importance Analysis

In [ ]:
def analyze_feature_importance(model, feature_names, model_name, top_n=20):
    """Analyze and visualize feature importance"""
    
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        
        # Create feature importance DataFrame
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print(f"\n🔍 {model_name} - Top {top_n} Most Important Features:")
        print("=" * 60)
        
        for i, row in feature_imp.head(top_n).iterrows():
            print(f"{row['feature']:30s} {row['importance']:.4f}")
        
        # Visualize top features
        plt.figure(figsize=(12, 8))
        top_features = feature_imp.head(top_n)
        
        sns.barplot(data=top_features, y='feature', x='importance')
        plt.title(f'{model_name} - Top {top_n} Feature Importances')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()
        
        return feature_imp
    else:
        print(f"⚠️ {model_name} does not support feature importance")
        return None

# Analyze feature importance for tree-based models
feature_importance_results = {}

for name, model in trained_models.items():
    if name in ['RandomForest', 'XGBoost', 'LightGBM']:
        importance_df = analyze_feature_importance(
            model, 
            X.columns.tolist(), 
            name, 
            top_n=15
        )
        feature_importance_results[name] = importance_df

## 🚀 Cloudflare Workers Deployment Package

In [ ]:
def create_advanced_deployment_package(best_model_name, model, scaler, feature_names):
    """Create deployment package for the best performing model"""
    
    # Get model performance
    best_result = next(r for r in evaluation_results if r['model'] == best_model_name)
    
    # Create metadata
    metadata = {
        'training_date': datetime.now().isoformat(),
        'model_type': best_model_name,
        'problem_type': 'classification',
        'target_classes': {'-1': 'DOWN', '0': 'HOLD', '1': 'UP'},
        'performance': {
            'overall_accuracy': best_result['overall_accuracy'],
            'direction_accuracy': best_result['direction_accuracy'],
            'win_rate': best_result['win_rate'],
            'total_trades': best_result['total_trades'],
            'avg_return_per_trade': best_result['avg_return_per_trade'],
            'sharpe_ratio': best_result['sharpe_ratio']
        },
        'features': {
            'count': len(feature_names),
            'names': feature_names,
            'scaling_required': True
        },
        'trading_info': {
            'prediction_threshold': 0.005,  # 0.5%
            'symbols': SYMBOLS,
            'recommendation': 'DEPLOY' if best_result['direction_accuracy'] > 0.60 else 'NEEDS_IMPROVEMENT'
        }
    }
    
    # Create JavaScript inference code
    js_inference = f'''
/**
 * Advanced Trading Model - {best_model_name} Classification
 * Generated: {datetime.now().isoformat()}
 * Direction Accuracy: {best_result['direction_accuracy']:.1%}
 */

// Model configuration
const MODEL_CONFIG = {{
    type: '{best_model_name}',
    directionAccuracy: {best_result['direction_accuracy']:.3f},
    features: {len(feature_names)},
    threshold: 0.005
}};

// Feature names (must match training order)
const FEATURE_NAMES = {feature_names};

export async function runAdvancedPrediction(ohlcv, env) {{
    try {{
        // 1. Calculate technical indicators
        const technicalFeatures = calculateTechnicalIndicators(ohlcv);
        
        // 2. Add market context
        const marketFeatures = await addMarketContext(technicalFeatures, env);
        
        // 3. Scale features
        const scaledFeatures = scaleFeatures(marketFeatures);
        
        // 4. Run model prediction
        const prediction = run{best_model_name}Prediction(scaledFeatures);
        
        // 5. Interpret results
        const result = {{
            predicted_direction: prediction > 0 ? 'UP' : prediction < 0 ? 'DOWN' : 'HOLD',
            confidence: Math.abs(prediction) / Math.max(Math.abs(prediction), 1),
            model_type: '{best_model_name}',
            direction_accuracy: {best_result['direction_accuracy']:.3f},
            recommendation: prediction !== 0 ? 'TRADE' : 'HOLD'
        }};
        
        return result;
        
    }} catch (error) {{
        console.error('Advanced prediction error:', error);
        return {{
            predicted_direction: 'HOLD',
            confidence: 0,
            error: error.message
        }};
    }}
}}

function calculateTechnicalIndicators(ohlcv) {{
    // Implementation for calculating all technical indicators
    // This would include RSI, MACD, Bollinger Bands, etc.
    
    const features = {{}};
    
    // RSI calculation
    features.RSI_14 = calculateRSI(ohlcv, 14);
    
    // MACD calculation
    const macd = calculateMACD(ohlcv);
    features.MACD = macd.macd;
    features.MACD_Signal = macd.signal;
    
    // Add all other technical indicators...
    
    return features;
}}

// Note: Full implementation would include all technical indicator calculations
// and the complete {best_model_name} prediction algorithm
'''
    
    # Save files
    with open('advanced_trading_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    with open('advanced_trading_inference.js', 'w') as f:
        f.write(js_inference)
    
    # Create deployment guide
    deployment_guide = f'''
# Advanced Trading Model Deployment Guide

## Model Performance
- **Model Type**: {best_model_name}
- **Direction Accuracy**: {best_result['direction_accuracy']:.1%}
- **Win Rate**: {best_result['win_rate']:.1%}
- **Total Trades**: {best_result['total_trades']}
- **Recommendation**: {metadata['trading_info']['recommendation']}

## Key Improvements
✅ Advanced feature engineering (technical indicators)
✅ Classification approach (UP/DOWN/HOLD)
✅ Tree-based models optimized for tabular data
✅ Trading-focused evaluation metrics

## Deployment Status
{'🟢 READY FOR DEPLOYMENT' if best_result['direction_accuracy'] > 0.60 else '🟡 NEEDS FURTHER IMPROVEMENT'}

Direction accuracy > 60% is generally required for profitable trading.
Current model: {best_result['direction_accuracy']:.1%}
'''
    
    with open('ADVANCED_DEPLOYMENT_GUIDE.md', 'w') as f:
        f.write(deployment_guide)
    
    print(f"📦 Advanced deployment package created!")
    print(f"   📊 Model: {best_model_name}")
    print(f"   🎯 Direction Accuracy: {best_result['direction_accuracy']:.1%}")
    print(f"   📁 Files: metadata, inference code, deployment guide")
    
    return metadata

# Create deployment package for best model
if evaluation_results:
    best_model_name = best_model['model']
    best_model_obj = trained_models[best_model_name]
    
    deployment_metadata = create_advanced_deployment_package(
        best_model_name,
        best_model_obj,
        scaler,
        X.columns.tolist()
    )
    
    print(f"\n🎉 Advanced training completed!")
    print(f"📈 Best model achieved {best_model['direction_accuracy']:.1%} direction accuracy")
    
    if best_model['direction_accuracy'] > 0.60:
        print(f"✅ Model exceeds 60% threshold - READY FOR PRODUCTION")
    else:
        print(f"⚠️  Model below 60% threshold - Consider more feature engineering")
        print(f"💡 Suggestions: More data, ensemble methods, or alternative approaches")
else:
    print("❌ No models were successfully trained")